# Etude d'un système de spheres dures par simulation moléculaire

ATTENTION: le notebook suivant dans son état actuel s'exécute en un peu plus de 40 heures sur un processeur intel i5-5200U (dual core, limité à 2GHz).

In [ ]:
import numpy,pandas,time
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
import os

## Equation d'état à basse densité -- Deuxième coefficient du viriel

Nous allons ici comparer la pression donnée par la simulation avec celle connue analytiquement. Nous réalisons une simulation d'un gaz de sphères dures de température $T=1$, de densité $n=0.0002$ et constitué de $N=108$ particules, placées initialement dans une boîte cubique comprenant 3x3x3 mailles fcc.

Nous travaillons dans le système d'unités où le rayon des sphères vaut $1$, la masse des particules vaut $1$ et la constante de Boltzmann $k=1$.

La pression doit s'exprimer en fonction de la densité comme suit

$$ \frac {P}{nT} = 1 + B_2 n + B_3 n^2 + \mathcal{O}(n^3) $$

Les deuxième et troisième coefficients du viriel sont donnés par

$$ B_2 = \frac{16\pi R^3}{3} = \frac {16\pi}{3} = 16.755 $$
$$ B_3 = \frac{160\pi^2 R^6}{9} = \frac{160\pi}{9} = 175.460 $$

Les termes correspondant valent

$$ B_2 n = 16.755 \times 0.0002 = 3.351 \times 10^{-3} $$
$$ B_3 n^2 = 175.460 \times 0.0002^2 = 7.018 \times 10^{-6} $$

Nous négligerons la contribution des coefficients d'ordre 3 et plus. 

L'équation d'état que nous devrions trouver numériquement pour la pression est donc 

$$ \frac {P}{nT} = 1.00335(1) $$

### Equation d'état à partir d'une longue simulation

In [ ]:
start = time.time()

os.system('./simulationO3 3 3 3 0.0002 1 20000 0')
os.system('mkdir data_eq_etat')
os.system("mv data/collisionData.csv data_eq_etat/collisionData_LD_long.csv")
os.system("mv data/infoSimulation.csv data_eq_etat/infoSimulation_LD_long.csv")
os.system("mv data/particle0Data.csv data_eq_etat/particle0Data_LD_long.csv")
os.system("mv data/excursionData.csv data_eq_etat/excursionData_LD_long.csv")

print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
collisionData = pandas.read_csv('data_eq_etat/collisionData_LD_long.csv')
[t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

print (t[-1])

In [ ]:
N = 108
T = 1
Delta_t = 100

p = numpy.ones(int((t[-1]-t[0])/Delta_t)+1) # pression*beta/densité
for i in range(len(t)):
    p[int((t[i]-t[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr[i]
    
p = p[:-1] # dernier bin incomplet

In [ ]:
pyplot.figure(figsize=(8,5));
pyplot.plot(p, '.');
pyplot.title("p pour chaque Delta_t");
pyplot.xlabel('t/Delta_t');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

In [ ]:
p_moy = numpy.sum(p)/len(p)
print (p_moy)

Nous obtenons une valeur proche de la prédiction. Nous estimons plus loin l'erreur sur ce résultat.

#### Estimation de l'erreur en considérant plusieurs les subdivisions $\Delta t$ de ce temps comme indépendantes

Attention: nous ne savons pas si cette hypothèse est valide, nous comparerons plus loin cette erreur à celle obtenue en considérant plusieurs simulations différentes.

In [ ]:
p_err = numpy.std(p)/numpy.sqrt(len(p))
print (p_err)

### Calcul du temps de corrélation

La détermination du temps de corrélation est importante pour connaître le durée $\Delta t$ à utiliser pour calculer la moyenne et l'erreur sur la pression du système. En effet, les valeurs de la pression calculées sur un intervalle $\Delta t$ trop petit sont corrélées entre elles ce qui complique le calcul de l'erreur statistique sur la pression moyenne. Le temps de corrélation $\tau$ d'une observable $A$ est défini à partir de la fonction de corrélation  $c_{AA}$:

$$ \tau = \frac 12 \sum_{n=-\infty}^{+\infty} \frac{c_{AA}(n)}{c_{AA}(0)} $$
$$ c_{AA}(k) = \langle A_n A_{n+k} \rangle - \langle A_n \rangle ^2 $$

L'observable en question est la "pression réduite" que l'on a défini comme:

$$ p_n = \frac{\beta P_n}{n} = 1 + \frac{1}{3 N T \Delta t} \sum v_{ij}\cdot r_{ij} \qquad \text{somme sur les collisions durant [}n\Delta t\text{ , } (n+1)\Delta t \text{[} $$

In [ ]:
def c_pp(k,p):
    
    N = len(p)
    
    if k>=0:
        avg1 = numpy.sum(p[0:N-k]*p[k:N]) / (N-k) # < p_n*p_{n+k} >
        avg2 = numpy.sum(p[0:N-k]) / (N-k) # < p_n >
    elif k<0:
        avg1 = numpy.sum(p[-k:N]*p[0:N+k]) / (N+k) # < p_n*p_{n+k} >
        avg2 = numpy.sum(p[-k:N]) / (N+k) # < p_n >
    
    return avg1-avg2**2

In [ ]:
N = 108
T = 1
Delta_t = 10

p = numpy.ones(int((t[-1]-t[0])/Delta_t)+1) # pression réduite
for i in range(len(t)):
    p[int((t[i]-t[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr[i]
    
p = p[:-1] # dernier bin incomplet

On vérifie que $c_{pp}(0)$ est bien la variance du vecteur des pressions réduites.

In [ ]:
print( c_pp(0,p) )
print( numpy.std(p)**2 )

<span style="color:red"> La fonction de corrélation semble beaucoup trop grande. Il me semble qu'elle est censée décroître quand $k$ s'éloigne de $0$. Est-ce donc une erreur ou alors le système est corrélé sur des temps très longs ? </span>

In [ ]:
for i in range(int(len(p)/2)):
    print( "c_pp({:d},p) = {:.3f}e-06".format(i,c_pp(i,p)*1e6) )

### Execution de plusieurs simulations pour avoir des données indépendantes

On exécute les commandes linux servant à effectuer les simulations avec le module "os".

In [ ]:
NSim = 20 # nombre de simulations

start = time.time()
for i in range(NSim):
    print ("./simulationO3 3 3 3 0.0002 1 1000 0")
    os.system("./simulationO3 3 3 3 0.0002 1 1000 0")
    os.system("mv data/collisionData.csv data_eq_etat/collisionData_LD_short{:d}.csv".format(i))
    os.system("mv data/infoSimulation.csv data_eq_etat/infoSimulation_LD_short{:d}.csv".format(i))
    os.system("mv data/particle0Data.csv data_eq_etat/particle0Data_LD_short{:d}.csv".format(i))
    os.system("mv data/excursionData.csv data_eq_etat/excursionData_LD_short{:d}.csv".format(i))
print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
N = 108
T = 1
t_sim = 1000

p_sim = numpy.zeros(NSim)

for i in range(NSim):
    
    collisionData = pandas.read_csv("data_eq_etat/collisionData_LD_short{:d}.csv".format(i))
    [t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

    p_sim[i] = 1 + 1/(N*3*T*t_sim) * numpy.sum(vDotr)

print (p_sim)

pyplot.figure(figsize=(8,5));
pyplot.plot(p_sim, '.');
pyplot.title("p pour plusieurs simulations");
pyplot.xlabel('simulation');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

p_moy = numpy.sum(p_sim)/len(p_sim)
p_err = numpy.std(p_sim)/numpy.sqrt(len(p_sim))
print (p_moy)
print (p_err)

Nous remarquons que la valeur obtenue est éloignée d'environ 1 sigma avec la prédiction théorique. L'accord est satisfaisant.

## Equation d'état à haute densité

### Equation d'état à partir d'une longue simulation

In [ ]:
n_max = 1/8*numpy.sqrt(2)
print (n_max)

In [ ]:
start = time.time()
os.system("./simulationO3 3 3 3 {:.8f} 1 200 0".format(0.3*n_max))
os.system("mv data/collisionData.csv data_eq_etat/collisionData_HD_long.csv".format(i))
os.system("mv data/infoSimulation.csv data_eq_etat/infoSimulation_HD_long.csv".format(i))
os.system("mv data/particle0Data.csv data_eq_etat/particle0Data_HD_long.csv".format(i))
os.system("mv data/excursionData.csv data_eq_etat/excursionData_HD_long.csv".format(i))
print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
collisionData = pandas.read_csv('data_eq_etat/collisionData_HD_long.csv')
[t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

N = 108
T = 1
Delta_t = 10

p = numpy.ones(int((t[-1]-t[0])/Delta_t)+1) # pression*beta/densité
for i in range(len(t)):
    p[int((t[i]-t[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr[i]
    
p = p[:-1] # dernier bin incomplet

pyplot.figure(figsize=(8,5));
pyplot.plot(p, '.');
pyplot.title("p pour chaque Delta_t");
pyplot.xlabel('t/Delta_t');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

p_moy = numpy.sum(p)/len(p)
print (p_moy)

### Execution de plusieurs simulations pour avoir des données indépendantes

In [ ]:
NSim = 20 # nombre de simulations

start = time.time()
for i in range(NSim):
    print ("./simulationO3 3 3 3 {:.8f} 1 10 0".format(0.3*n_max))
    os.system("./simulationO3 3 3 3 {:.8f} 1 10 0".format(0.3*n_max))
    os.system("mv data/collisionData.csv data_eq_etat/collisionData_HD_short{:d}.csv".format(i))
    os.system("mv data/infoSimulation.csv data_eq_etat/infoSimulation_HD_short{:d}.csv".format(i))
    os.system("mv data/particle0Data.csv data_eq_etat/particle0Data_HD_short{:d}.csv".format(i))
    os.system("mv data/excursionData.csv data_eq_etat/excursionData_HD_short{:d}.csv".format(i))
print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
N = 108
T = 1
t_sim = 10

p_sim = numpy.zeros(NSim)

for i in range(NSim):
    
    collisionData = pandas.read_csv("data_eq_etat/collisionData_HD_short{:d}.csv".format(i))
    [t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

    p_sim[i] = 1 + 1/(N*3*T*t_sim) * numpy.sum(vDotr)

print (p_sim)

pyplot.figure(figsize=(8,5));
pyplot.plot(p_sim, '.');
pyplot.title("p pour plusieurs simulations");
pyplot.xlabel('simulation');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

p_moy = numpy.sum(p_sim)/len(p_sim)
p_err = numpy.std(p_sim)/numpy.sqrt(len(p_sim))
print (p_moy)
print (p_err)

## Pression en fonction de la densité

In [ ]:
n_max = 1/8*numpy.sqrt(2)
n = numpy.linspace(0.01*n_max,0.1*n_max,10)

os.system("mkdir data_pFctn")
start = time.time()
for i in range(len(n)):
    print ("./simulationO3 3 3 3 {:.8f} 1 100 0".format(n[i]))
    os.system("./simulationO3 3 3 3 {:.8f} 1 100 0".format(n[i]))
    os.system("mv data/collisionData.csv data_pFctn/collisionData{:.4f}.csv".format(n[i]/n_max))
    os.system("mv data/infoSimulation.csv data_pFctn/infoSimulation{:.4f}.csv".format(n[i]/n_max))
    os.system("mv data/particle0Data.csv data_pFctn/particle0Data{:.4f}.csv".format(n[i]/n_max))
    os.system("mv data/excursionData.csv data_pFctn/excursionData{:.4f}.csv".format(n[i]/n_max))
print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
n_max = 1/8*numpy.sqrt(2)
n = numpy.linspace(0.5*n_max,0.8*n_max,61)

start = time.time()
for i in range(len(n)):
    print ("./simulationO3 3 3 3 {:.8f} 1 200 0".format(n[i]))
    os.system("./simulationO3 3 3 3 {:.8f} 1 200 0".format(n[i]))
    os.system("mv data/collisionData.csv data_pFctn/collisionData{:.4f}.csv".format(n[i]/n_max))
    os.system("mv data/infoSimulation.csv data_pFctn/infoSimulation{:.4f}.csv".format(n[i]/n_max))
    os.system("mv data/particle0Data.csv data_pFctn/particle0Data{:.4f}.csv".format(n[i]/n_max))
    os.system("mv data/excursionData.csv data_pFctn/excursionData{:.4f}.csv".format(n[i]/n_max))
print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
#n = numpy.linspace(0.01*n_max,0.1*n_max,10)
n = numpy.linspace(0.5*n_max,0.8*n_max,61)
p_n = numpy.zeros(len(n))

N = 108
T = 1
Delta_t = 10

for i in range(len(n)):
    
    collisionData = pandas.read_csv("data_pFctn/collisionData{:.4f}.csv".format(n[i]/n_max))
    [t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

    p = numpy.ones(int((t[-1]-t[0])/Delta_t)+1) # pression réduite
    for j in range(len(t)):
        p[int((t[j]-t[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr[j]

    p = p[10:-1] # dernier bin incomplet, prend qu'à partir de 10 (semble avoir atteint un équilibre)
    
    p_n[i] = numpy.sum(p)/len(p)

pyplot.figure(figsize=(8,5));
pyplot.plot(n/n_max,p_n, '.');
pyplot.title("p en fonction de n");
pyplot.xlabel('n/n_max');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

pyplot.figure(figsize=(8,5));
pyplot.plot(n/n_max,p_n*n/n_max, '.');
pyplot.title("P en fonction de n");
pyplot.xlabel('n/n_max');
pyplot.ylabel('P/(n_max*k*T)');
pyplot.grid();
pyplot.show()

## "Fluidité" en fonction de la densité

Nous étudions ici la fluidité du système en fonction de sa densité. Comme indicateur de la fluidité, nous utiliserons le temps mis par une particule pour sortir d'une sphère de rayon 1 centrée autour de sa position initiale. Plus ce temps est long, plus la particule a du mal à se mouvoir entre les autres particules ce qui indique que le système est moins fluide. La transition fluide-solide devrait être caractérisée par une augmentation considérable de ce temps de sortie. 

Nous appelerons "fluidité" l'inverse de ce temps de sortie. Nous calculons la fluidité pour chacune des particules et définissons la fluidité du système comme la moyenne des fluidités individuelles. Le temps de sortie du système est donc la moyenne harmonique des temps de sortie individuels.

In [ ]:
n_max = 1/8*numpy.sqrt(2)
n = numpy.linspace(0.55*n_max,0.7*n_max,31)

os.system("mkdir data_fluidity")
start = time.time()
for i in range(len(n)):
    print ("./simulationO3 3 3 3 {:.8f} 1 50 0".format(n[i]))
    os.system("./simulationO3 3 3 3 {:.8f} 1 50 0".format(n[i]))
    os.system("mv data/collisionData.csv data_fluidity/collisionData{:.4f}.csv".format(n[i]/n_max))
    os.system("mv data/excursionData.csv data_fluidity/excursionData{:.4f}.csv".format(n[i]/n_max))
    os.system("mv data/infoSimulation.csv data_fluidity/infoSimulation{:.4f}.csv".format(n[i]/n_max))
    os.system("mv data/particle0Data.csv data_fluidity/particle0Data{:.4f}.csv".format(n[i]/n_max))
print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
n = numpy.linspace(0.55*n_max,0.7*n_max,31)
fluidity_n = numpy.zeros(len(n))

for i in range(len(n)):
    
    excursionData = pandas.read_csv("data_fluidity/excursionData{:.4f}.csv".format(n[i]/n_max))
    [fluidity] = numpy.transpose(excursionData.as_matrix(['fluidity']))
    infoSimulation = pandas.read_csv("data_fluidity/infoSimulation{:.4f}.csv".format(n[i]/n_max))
    [mx,my,mz,a] = numpy.transpose(infoSimulation.as_matrix(['mx','my','mz','a']))
    
    fluidity_n[i] = numpy.sum(fluidity)/len(fluidity) # moyenne des fluidités de chaque particule

pyplot.figure(figsize=(8,5));
pyplot.plot(n/n_max,fluidity_n, '.'); 
pyplot.title("fluidité en fonction de n");
pyplot.xlabel('n/n_max');
pyplot.ylabel('fluidité');
pyplot.grid();
pyplot.show()

pyplot.figure(figsize=(8,5));
pyplot.plot(n/n_max,numpy.log(1/fluidity_n), '.'); # risque de division par 0 mais on s'en fiche
#pyplot.plot(n/n_max,1/fluidity_n, '.');
pyplot.title("temps de sortie en fonction de n");
pyplot.xlabel('n/n_max');
pyplot.ylabel('log temps de sortie');
pyplot.grid();
pyplot.show()

Nous remarquons que le système perd toute fluidité et se fige au point de transition ($n=0.6\,n_{max}$) observé avec le graphique $p(n)$. Cela correspond à nos attentes et fourni une deuxième confirmation du point de transition. 